# [하이퍼튜닝](https://tykimos.github.io/2019/05/10/KerasTuner/)

# Keras Tuner

Keras Tuner는 텐서플로에서 최적의 하이퍼파라미터 세트를 선택하는데 도움을 주는 라이브러리입니다. 머신러닝 애플리케이션에서 올바른 하이퍼파라미터 세트를 선택하는 과정을 *하이퍼파라미터 조정* 또는 *하이퍼튜닝*이라고 합니다.

하이퍼파라미터는 훈련 프로세스 및 ML 모델의 토폴로지를 제어하는 변수입니다. 이러한 변수는 훈련 과정에서 일정하게 유지되며 ML 프로그램의 성능에 직접적으로 영향을 미칩니다. 하이퍼파라미터에는 두 가지 유형이 있습니다.

1. **레이어의 설정 및 레이어의 구조**와 같은 하이퍼파라미터
2. SGD(Stochastic Gradient Descent)의 **학습률** 및 KNN(k Nearest Neighbors) 분류자의 **최근접 이웃 수**와 같은 학습 알고리즘의 속도와 품질에 영향을 주는 알고리즘 하이퍼파라미터

이 튜토리얼에서는 Keras Tuner를 사용하여 이미지 분류 애플리케이션에 하이퍼튜닝을 수행합니다.



---


## 1. 설정

In [ ]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [ ]:
!pip install -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 102kB 5.7MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-cp37-none-any.whl size=9569 sha256=f9bfcba9a1bbff870376e933c58ece046597fa6edc5084f6d58ffb265b082be3
  Stored in directory: /root/.cache/pip/wheels/95/99/a8/6b116b0f69be60cc475d1cd36680f3a09f284d86655bb99d93
Successfully built kt-legacy


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  




---


## 2. 데이터세트 다운로드 및 준비하기

Fashion MNIST 데이터세트에서 의류 이미지를 분류하는 머신러닝 모델에 가장 적합한 하이퍼파라미터를 찾아보겠습니다.

데이터를 로드합니다.

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
# 0~255 범위로 표기된 픽셀을 0~1 범위로 변환합니다.
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0



---


## 3. 모델 정의하기

하이퍼튜닝과 관련된 모델을 만들 때 레이어와 더불어 하이퍼파라미터 탐색 범위도 지정합니다. 하이퍼튜닝과 관련된 모델을 *하이퍼 모델*이라고 합니다.

모델 빌더 함수를 만들어 하이퍼 모델을 정의해봅시다.

In [ ]:
def model_builder(hp):  # 하이퍼모델
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    # 다음 층에 적용할 유닛 수 하이퍼파라미터 탐색 범위 지정하기
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units = hp_units, activation='relu'))  # 이 층에 탐색 범위 적용
    model.add(keras.layers.Dense(10))
    # 옵티마이저에 적용할 학습률 하이퍼파라미터 탐색 범위 지정하기
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),  # 옵티마이저에 학습률 탐색 범위 적용
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                  metrics = ['accuracy'])
    return model



---


## 4. 튜너를 인스턴스화하고 하이퍼튜닝 수행하기

Keras Tuner에는 `RandomSearch`, `Hyperband`, `BayesianOptimization`, `Sklearn`라는 네 가지 튜너가 있습니다. 우리는 이 중에서 Hyperband를 사용합니다.

Hyperband 튜너를 인스턴스화하려면 최적화할 하이퍼모델인 `objective`, 훈련할 최대 에포크 수인 `max_epochs`를 지정해야합니다. 

위에서 만든 모델 빌더 함수를 넣고 튜너를 만듭니다.

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

Hyperband 튜닝 알고리즘은 몇 개의 epoch에 대해 많은 수의 모델을 훈련하고 최고 성능을 보이는 절반만 다음 단계로 넘깁니다. Hyperband는 1 + log<sub><code>factor</code></sub>( `max_epochs`)를 계산하고 이를 가장 가까운 정수로 반올림하여 한 브래킷에서 훈련할 모델 수를 결정합니다.

하이퍼파라미터 검색을 실행하기 전에 훈련 단계가 끝날 때마다 훈련 결과를 지우도록 콜백을 정의합니다.


In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

이제 하이퍼파라미터 검색을 실행합니다. 검색 메서드의 인수는 위의 콜백 외에 `tf.keras.model.fit`에 사용되는 인수와 같습니다.

검색 실행은 모델을 훈련하는 것과 같습니다.

최적의 하이퍼파라미터 `best_hps`에는 **유닛 수와 학습률**이 들어있습니다.

In [ ]:
tuner.search(img_train, label_train, epochs=10, 
             validation_data = (img_test, label_test), 
             callbacks = [ClearTrainingOutput()])
# 최적의 하이퍼파라미터를 뽑아낸다.
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 02m 22s]
val_accuracy: 0.8730000257492065

Best val_accuracy So Far: 0.885200023651123
Total elapsed time: 00h 16m 35s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimizer
is 0.001.



이제 최적의 하이퍼파라미터로 모델을 재훈련합니다.

In [ ]:
# 최적 하이퍼파라미터 값을 넣은 튜너의 하이퍼모델을 새로운 모델로 만든다.
# 이후 모델을 재훈련한다.
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs=10,
          validation_data=(img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4754 - accuracy: 0.8307 - val_loss: 0.4378 - val_accuracy: 0.8365
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3589 - accuracy: 0.8684 - val_loss: 0.4115 - val_accuracy: 0.8554
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3235 - accuracy: 0.8816 - val_loss: 0.3489 - val_accuracy: 0.8738
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2970 - accuracy: 0.8906 - val_loss: 0.3612 - val_accuracy: 0.8713
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2816 - accuracy: 0.8956 - val_loss: 0.3378 - val_accuracy: 0.8798
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2669 - accuracy: 0.9002 - val_loss: 0.3384 - val_accuracy: 0.8833
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2524 - accuracy: 0.9054 - val_loss: 0.3702 - val_accuracy:

`my_dir/intro_to_kt` 디렉토리에는 하이퍼파라미터 검색 중에 실행되는 모든 시험(모델 구성)에 대한 상세 로그와 체크포인트가 들어 있습니다. 하이퍼파라미터 검색을 다시 실행하면 Keras Tuner가 이러한 로그의 기존 상태를 사용하여 검색을 재개합니다. 